In [1]:
import cv2
import numpy as np
import time

If the code was taken from github, then run this block to get to correct directory

In [2]:
%cd ../
%cd weights

c:\Users\perez\OneDrive\Documents\GitHub\Yolo
c:\Users\perez\OneDrive\Documents\GitHub\Yolo\weights


In [3]:
net = cv2.dnn.readNet("yolov3_tiny_5_gestures.weights", "yolov3-tiny-custom.cfg")
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [4]:
layer_names = net.getLayerNames()

Depending on the version of CV and NumPy you might have to run this code differently. Run the code below and if it errors out run the other code.

In [5]:
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

In [6]:
holder = net.getUnconnectedOutLayers()
output_layers = []
for i in holder:
    output_layers.append(layer_names[int(i-1)])

In [7]:
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [13]:
cap = cv2.VideoCapture(1)
cap.set(3,640)
cap.set(4,480)
font = cv2.FONT_HERSHEY_PLAIN
frame_id = 1
if not cap.isOpened():
    raise ValueError("Error: Video capture not initialized.")

In [14]:
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [15]:

while True:
    starting_time = time.time()
    _, frame = cap.read()
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (64, 64), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.80:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                print(class_id)
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1, color, 2)  

    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    print(fps)
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 4, (0, 0, 0), 3)
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    #frame_id = frame_id + 1
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

1.9017957381824249
136.5955839249658
99.5845956598129
79.90368056084736
249.0531441125824
38.618027805911055
40.79784450475162
29.56858653507226
81.64095377128953
398.69809885931556
59.41529613417761
52.63407288424857
28.232113889543296
52.17834394904459
33.652427869957314
46.21975382106295
249.88406315162348
30.742816934443532
71.29411364756676
42.216603591271436
42.53124714805764
265.6977068288357
56.43497800083422
34.89321486805763
19.086533910954167
60.53085493274837
21.418087116376448
35.468001623596265
238.6517211948791
41.96192286528938
43.161932987568946
21.493379248144958
40.97720722569048
227.96369367900428
41.70573437142658
41.170678079233575
28.450425640156013
71.39726960133456
23.17679173343648
32.99951220279775
222.2854417298214
83.34434177844014
77.69099968510938
26.17481059896906
39.29605756258432
216.81592142672525
29.00544936516279
43.40761285782295
26.323808328364766
37.272762818803876
21.145867679013467
40.247416349207874
304.796453746094
23.679107553166865
25.33098

KeyboardInterrupt: 

In [16]:
cap.release()
cv2.destroyAllWindows()